# Test MITMProxy

#### Imports Basics

In [3]:
import pytest
import pandas as pd
import os
import typing
import contextlib
from unittest import mock

#### Imports MITM

In [4]:
import mitmproxy.exceptions
import mitmproxy.types
from mitmproxy.test import taddons
from mitmproxy.test import tflow
from mitmproxy import command
from mitmproxy import flow

#### Import Test

In [5]:
from . import test_command

ImportError: cannot import name 'test_command' from '__main__' (unknown location)

***

### Só Quero Chorar

In [6]:
#import os
from mitmproxy import controller, proxy
from mitmproxy.proxy.server import ProxyServer

from mitmproxy import proxy, options
from mitmproxy.tools.dump import DumpMaster
from mitmproxy.addons import core

In [7]:
class AddHeader(controller.Channel):
    def __init__(self, server):
        controller.Master.__init__(self, server)
        self.num = 0
        self.stickyhosts = {}      
           
    def response(self, flow):
        self.num = self.num + 1
        print(self.num)
        flow.response.headers["count"] = str(self.num)
        
    def run(self):
        try:
            return controller.Channel.run(self)
        except KeyboardInterrupt:
            self.shutdown()
            
    @controller.handler
    def request(self, flow):
        hid = (flow.request.host, flow.request.port)
        if "cookie" in flow.request.headers:
            self.stickyhosts[hid] = flow.request.headers.get_all("cookie")
        elif hid in self.stickyhosts:
            flow.request.headers.set_all("cookie", self.stickyhosts[hid])

    @controller.handler
    def response(self, flow):
        hid = (flow.request.host, flow.request.port)
        if "set-cookie" in flow.response.headers:
            self.stickyhosts[hid] = flow.response.headers.get_all("set-cookie")
        
        


addons = [
    AddHeader()
]

opts = options.Options(listen_host='127.0.0.1', listen_port=8080)
pconf = proxy.config.ProxyConfig(opts)

m = DumpMaster(None)
##m = StickyMaster(server)
m.server = proxy.server.ProxyServer(pconf)
# print(m.addons)
m.addons.add(addons)
print(m.addons)
# m.addons.add(core.Core())

try:
    m.run()
    ##m.run()
except KeyboardInterrupt:
    m.shutdown()

AttributeError: module 'mitmproxy.controller' has no attribute 'handler'

***

In [9]:
class AddHeader:
    def __init__(self):
        self.num = 0

    def response(self, flow):
        self.num = self.num + 1
        print(self.num)
        flow.response.headers["count"] = str(self.num)


addons = [
    AddHeader()
]

opts = options.Options(listen_host='127.0.0.1', listen_port=8089)
pconf = proxy.config.ProxyConfig(opts)

m = DumpMaster(None)
##m = StickyMaster(server)
m.server = proxy.server.ProxyServer(pconf)
print(m.addons)
m.addons.add(addons)
print(m.addons)
#m.addons.add(core.Core())

try:
    m.run()
except KeyboardInterrupt:
    m.shutdown()

['<mitmproxy.addons.termlog.TermLog object at 0x7f00a8598550>',
 '<mitmproxy.addons.termstatus.TermStatus object at 0x7f00a8598588>',
 '<mitmproxy.addons.core.Core object at 0x7f00a85985c0>',
 '<mitmproxy.addons.browser.Browser object at 0x7f00a85985f8>',
 '<mitmproxy.addons.block.Block object at 0x7f00a8598668>',
 '<mitmproxy.addons.anticache.AntiCache object at 0x7f00a8598630>',
 '<mitmproxy.addons.anticomp.AntiComp object at 0x7f00a85986d8>',
 '<mitmproxy.addons.check_ca.CheckCA object at 0x7f00a8598710>',
 '<mitmproxy.addons.clientplayback.ClientPlayback object at 0x7f00a8598748>',
 '<mitmproxy.addons.cut.Cut object at 0x7f00a8598860>',
 '<mitmproxy.addons.disable_h2c.DisableH2C object at 0x7f00a8598898>',
 '<mitmproxy.addons.export.Export object at 0x7f00a85988d0>',
 '<mitmproxy.addons.onboarding.Onboarding object at 0x7f00a8598908>',
 '<mitmproxy.addons.proxyauth.ProxyAuth object at 0x7f00a8598978>',
 '<mitmproxy.addons.replace.Replace object at 0x7f00a85989e8>',
 '<mitmproxy.add

RuntimeError: Cannot close a running event loop

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


***

#### Script Kiddie

In [1]:
from mitmproxy import command
from mitmproxy import ctx

In [2]:
class MyAddon:
    def __init__(self):
        self.num = 0

    @command.command("myaddon.inc")
    def inc(self) -> None:
        self.num += 1
        ctx.log.info("num = %s" % self.num)


addons = [
    MyAddon()
]

***

#### Def Study

In [1]:
class AddHeader:
    def response(self, flow):
        flow.response.headers["newheader"] = "foo"


def start():
    return AddHeader()

In [12]:
@contextlib.contextmanager
def chdir(path: str):
    old_dir = os.getcwd()
    os.chdir(path)
    yield
    os.chdir(old_dir)

In [13]:
def test_bool():
    with taddons.context() as tctx:
        b = mitmproxy.types._BoolType()
        assert b.completion(tctx.master.commands, bool, "b") == ["false", "true"]
        assert b.parse(tctx.master.commands, bool, "true") is True
        assert b.parse(tctx.master.commands, bool, "false") is False
        assert b.is_valid(tctx.master.commands, bool, True) is True
        assert b.is_valid(tctx.master.commands, bool, "foo") is False
        with pytest.raises(mitmproxy.exceptions.TypeError):
            b.parse(tctx.master.commands, bool, "foo")

In [14]:
def test_str():
    with taddons.context() as tctx:
        b = mitmproxy.types._StrType()
        assert b.is_valid(tctx.master.commands, str, "foo") is True
        assert b.is_valid(tctx.master.commands, str, 1) is False
        assert b.completion(tctx.master.commands, str, "") == []
        assert b.parse(tctx.master.commands, str, "foo") == "foo"

In [15]:
def test_unknown():
    with taddons.context() as tctx:
        b = mitmproxy.types._UnknownType()
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Unknown, "foo") is False
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Unknown, 1) is False
        assert b.completion(tctx.master.commands, mitmproxy.types.Unknown, "") == []
        assert b.parse(tctx.master.commands, mitmproxy.types.Unknown, "foo") == "foo"

In [16]:
def test_int():
    with taddons.context() as tctx:
        b = mitmproxy.types._IntType()
        assert b.is_valid(tctx.master.commands, int, "foo") is False
        assert b.is_valid(tctx.master.commands, int, 1) is True
        assert b.completion(tctx.master.commands, int, "b") == []
        assert b.parse(tctx.master.commands, int, "1") == 1
        assert b.parse(tctx.master.commands, int, "999") == 999
        with pytest.raises(mitmproxy.exceptions.TypeError):
            b.parse(tctx.master.commands, int, "foo")

In [18]:
def test_cmd():
    with taddons.context() as tctx:
        tctx.master.addons.add(test_command.TAddon())
        b = mitmproxy.types._CmdType()
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Cmd, "foo") is False
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Cmd, "cmd1") is True
        assert b.parse(tctx.master.commands, mitmproxy.types.Cmd, "cmd1") == "cmd1"
        with pytest.raises(mitmproxy.exceptions.TypeError):
            assert b.parse(tctx.master.commands, mitmproxy.types.Cmd, "foo")
        assert len(
            b.completion(tctx.master.commands, mitmproxy.types.Cmd, "")
        ) == len(tctx.master.commands.commands.keys())

In [20]:
def test_arg():
    with taddons.context() as tctx:
        b = mitmproxy.types._ArgType()
        assert b.completion(tctx.master.commands, mitmproxy.types.Arg, "") == []
        assert b.parse(tctx.master.commands, mitmproxy.types.Arg, "foo") == "foo"
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Arg, "foo") is True
        assert b.is_valid(tctx.master.commands, mitmproxy.types.Arg, 1) is False

In [21]:
def test_strseq():
    with taddons.context() as tctx:
        b = mitmproxy.types._StrSeqType()
        assert b.completion(tctx.master.commands, typing.Sequence[str], "") == []
        assert b.parse(tctx.master.commands, typing.Sequence[str], "foo") == ["foo"]
        assert b.parse(tctx.master.commands, typing.Sequence[str], "foo,bar") == ["foo", "bar"]
        assert b.is_valid(tctx.master.commands, typing.Sequence[str], ["foo"]) is True
        assert b.is_valid(tctx.master.commands, typing.Sequence[str], ["a", "b", 3]) is False
        assert b.is_valid(tctx.master.commands, typing.Sequence[str], 1) is False
        assert b.is_valid(tctx.master.commands, typing.Sequence[str], "foo") is False

In [27]:
def test_typemanager():
    assert mitmproxy.types.CommandTypes.get(bool, None)
    assert mitmproxy.types.CommandTypes.get(mitmproxy.types.Choice("choide"), None)

***

#### INSTALLED VERSIONS

In [10]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.7.2.final.0
python-bits: 64
OS: Linux
OS-release: 4.18.0-16-generic
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: pt_BR.UTF-8
LOCALE: pt_BR.UTF-8

pandas: 0.24.2
pytest: 4.3.1
pip: 19.0.3
setuptools: 40.8.0
Cython: None
numpy: 1.16.2
scipy: None
pyarrow: None
xarray: None
IPython: 7.3.0
sphinx: None
patsy: None
dateutil: 2.8.0
pytz: 2018.9
blosc: None
bottleneck: None
tables: None
numexpr: None
feather: None
matplotlib: None
openpyxl: None
xlrd: None
xlwt: None
xlsxwriter: None
lxml.etree: 4.3.2
bs4: None
html5lib: None
sqlalchemy: None
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
gcsfs: None


***